XML 관계 데이터 생성 함수 정의

In [ ]:
# ------------------------------------------------------
# XML 기반 직원 관계 데이터 생성 시스템
# ------------------------------------------------------

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from xml.dom import minidom
from pathlib import Path
import json
from datetime import datetime

# 설정
DATA_PATH = Path("../data/IBM_HR_personas_assigned.csv")
XML_OUTPUT_PATH = Path("../data/employee_relationships.xml")
JSON_OUTPUT_PATH = Path("../data/employee_relationships.json")  # JSON 버전도 생성
RNG_SEED = 42
np.random.seed(RNG_SEED)

def prettify_xml(elem):
    """XML을 보기 좋게 포매팅"""
    rough_string = ET.tostring(elem, 'unicode')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

print("XML 기반 관계 데이터 생성 시스템 초기화 완료")

관계 생성 로직 함수들

In [ ]:
# ------------------------------------------------------
# 개인차가 반영된 페르소나 기반 관계 생성 로직 (모든 함수)
# ------------------------------------------------------

def get_persona_bias(persona_code, employee_id=None, trait_type='collab_strength'):
    """페르소나별 편향값 + 개인차 반영"""
    
    # 기본 페르소나 편향 (중심값)
    base_configs = {
        # 고위험군
        'P01': {'collab_strength': 0.7, 'frequency': 0.6, 'quality_poor_prob': 0.4, 'leadership_prob': 0.2},
        'P02': {'collab_strength': 0.5, 'frequency': 0.4, 'quality_poor_prob': 0.5, 'leadership_prob': 0.1}, 
        'P03': {'collab_strength': 0.9, 'frequency': 0.8, 'quality_poor_prob': 0.2, 'leadership_prob': 0.5},
        'P04': {'collab_strength': 1.0, 'frequency': 0.9, 'quality_poor_prob': 0.1, 'leadership_prob': 0.7},
        
        # 안정군
        'S01': {'collab_strength': 1.1, 'frequency': 1.0, 'quality_poor_prob': 0.1, 'leadership_prob': 0.6},
        'S02': {'collab_strength': 1.3, 'frequency': 1.2, 'quality_poor_prob': 0.05, 'leadership_prob': 0.8},
        'S03': {'collab_strength': 1.1, 'frequency': 0.9, 'quality_poor_prob': 0.1, 'leadership_prob': 0.5},
        
        # 중립군
        'N01': {'collab_strength': 0.8, 'frequency': 0.7, 'quality_poor_prob': 0.3, 'leadership_prob': 0.3},
        'N02': {'collab_strength': 0.9, 'frequency': 0.6, 'quality_poor_prob': 0.4, 'leadership_prob': 0.6}, 
        'N03': {'collab_strength': 1.0, 'frequency': 0.8, 'quality_poor_prob': 0.2, 'leadership_prob': 0.4}
    }
    
    base_config = base_configs.get(persona_code, base_configs['N01'])
    
    # 개인별 특성이 필요한 경우
    if employee_id is not None and trait_type is not None:
        return get_individual_trait(persona_code, employee_id, trait_type, base_config)
    
    return base_config

def get_individual_trait(persona_code, employee_id, trait_type, base_config):
    """개인별 특성 생성 - 페르소나 중심값 + 개인차"""
    
    # 개인별 일관된 랜덤 시드 (같은 직원은 항상 같은 특성 유지)
    individual_seed = RNG_SEED + int(employee_id) + hash(trait_type) % 10000
    np.random.seed(individual_seed)
    
    base_value = base_config[trait_type]
    
    # 개인별 페르소나 강도 (0.5~1.5 범위)
    persona_strength = np.random.uniform(0.5, 1.5)
    
    # 개인적 변동성 (±25%)
    individual_variation = np.random.uniform(0.75, 1.25)
    
    if trait_type in ['collab_strength', 'frequency', 'leadership_prob']:
        adjusted_value = base_value * persona_strength * individual_variation
        
        if trait_type in ['frequency', 'leadership_prob']:
            adjusted_value = max(0.1, min(1.5, adjusted_value))
        else:  # collab_strength
            adjusted_value = max(0.2, min(1.8, adjusted_value))
            
    else:  # quality_poor_prob
        logit_base = np.log(base_value / (1 - base_value))
        adjusted_logit = logit_base * persona_strength + np.random.normal(0, 0.5)
        adjusted_value = 1 / (1 + np.exp(-adjusted_logit))
    
    # 시드 리셋
    np.random.seed(RNG_SEED)
    
    return adjusted_value

def apply_persona_influence(base_value, persona_influence, randomness_factor=0.3):
    """페르소나 영향을 확률적으로 적용"""
    
    if np.random.random() < randomness_factor:
        return np.random.uniform(0.3, 1.5) * base_value
    else:
        return persona_influence * base_value

def generate_hierarchy_relationships(df):
    """개인차가 반영된 계층 관계 생성"""
    hierarchy_rels = {}
    
    for dept in df['Department'].unique():
        dept_employees = df[df['Department'] == dept].copy()
        dept_employees = dept_employees.sort_values(['JobLevel', 'YearsAtCompany'], 
                                                  ascending=[False, False])
        
        for high_level in [4, 5]:
            managers = dept_employees[dept_employees['JobLevel'] == high_level]
            subordinates = dept_employees[dept_employees['JobLevel'] == high_level-1]
            
            if len(managers) == 0 or len(subordinates) == 0:
                continue
                
            for _, manager in managers.iterrows():
                mgr_id = str(manager['EmployeeNumber'])
                mgr_persona = manager.get('argmax_Persona_Code', 'N01')
                
                # 개인별 리더십 성향 가져오기
                mgr_leadership = get_persona_bias(mgr_persona, mgr_id, 'leadership_prob')
                mgr_quality_prob = get_persona_bias(mgr_persona, mgr_id, 'quality_poor_prob')
                
                # 관리 스타일 결정 - 개인차 반영
                leadership_with_randomness = apply_persona_influence(1.0, mgr_leadership, 0.2)
                
                if leadership_with_randomness > 0.6:
                    mgmt_styles = ['collaborative', 'supportive']
                elif leadership_with_randomness < 0.3:
                    mgmt_styles = ['directive', 'supportive'] 
                else:
                    mgmt_styles = ['collaborative', 'directive', 'supportive']
                
                # 관리 범위 결정 - 리더십 성향의 개인차 반영
                base_span = np.random.randint(3, 8)
                if leadership_with_randomness > 0.7:
                    span_of_control = min(base_span + np.random.randint(1, 3), len(subordinates))
                elif leadership_with_randomness < 0.3:
                    span_of_control = max(1, base_span - np.random.randint(1, 3))
                else:
                    span_of_control = min(base_span, len(subordinates))
                
                selected_subs = subordinates.sample(
                    n=min(span_of_control, len(subordinates)),
                    random_state=RNG_SEED + manager['EmployeeNumber']
                )
                
                if mgr_id not in hierarchy_rels:
                    hierarchy_rels[mgr_id] = {'manages': [], 'reports_to': None}
                
                for _, sub in selected_subs.iterrows():
                    sub_id = str(sub['EmployeeNumber'])
                    
                    hierarchy_rels[mgr_id]['manages'].append({
                        'employee_id': sub_id,
                        'employee_name': f"Employee_{sub_id}",
                        'job_role': sub['JobRole'],
                        'job_level': int(sub['JobLevel']),
                        'relationship_start': '2023-01-01',
                        'management_style': np.random.choice(mgmt_styles),
                        'span_of_control': span_of_control
                    })
                    
                    if sub_id not in hierarchy_rels:
                        hierarchy_rels[sub_id] = {'manages': [], 'reports_to': None}
                    
                    # 보고 빈도 - 관계 품질의 개인차 반영
                    reporting_frequencies = ['daily', 'weekly', 'bi-weekly']
                    quality_influence = apply_persona_influence(mgr_quality_prob, 1.0, 0.25)
                    
                    if quality_influence > 0.4:  # 관계가 좋지 않으면 더 자주 보고
                        freq_weights = [0.4, 0.5, 0.1]
                    else:
                        freq_weights = [0.2, 0.6, 0.2]
                    
                    hierarchy_rels[sub_id]['reports_to'] = {
                        'manager_id': mgr_id,
                        'manager_name': f"Employee_{mgr_id}",
                        'manager_role': manager['JobRole'],
                        'manager_level': int(manager['JobLevel']),
                        'reporting_since': '2023-01-01',
                        'reporting_frequency': np.random.choice(reporting_frequencies, p=freq_weights)
                    }
    
    return hierarchy_rels

def generate_collaboration_relationships(df):
    """개인차가 반영된 협업 관계 생성"""
    collab_rels = {}
    
    for dept in df['Department'].unique():
        dept_employees = df[df['Department'] == dept]
        
        for i, emp1 in dept_employees.iterrows():
            emp1_id = str(emp1['EmployeeNumber'])
            emp1_persona = emp1.get('argmax_Persona_Code', 'N01')
            
            if emp1_id not in collab_rels:
                collab_rels[emp1_id] = []
            
            for j, emp2 in dept_employees.iterrows():
                if i >= j:
                    continue
                
                emp2_id = str(emp2['EmployeeNumber'])
                emp2_persona = emp2.get('argmax_Persona_Code', 'N01')
                
                # 개인별 협업 성향 가져오기
                emp1_collab = get_persona_bias(emp1_persona, emp1_id, 'collab_strength')
                emp2_collab = get_persona_bias(emp2_persona, emp2_id, 'collab_strength')
                emp1_freq = get_persona_bias(emp1_persona, emp1_id, 'frequency')
                emp2_freq = get_persona_bias(emp2_persona, emp2_id, 'frequency')
                emp1_quality = get_persona_bias(emp1_persona, emp1_id, 'quality_poor_prob')
                emp2_quality = get_persona_bias(emp2_persona, emp2_id, 'quality_poor_prob')
                
                # 기본 협업 강도 계산
                level_diff = abs(emp1['JobLevel'] - emp2['JobLevel'])
                base_strength = max(0.2, 1.0 - level_diff * 0.15)
                
                # 페르소나 영향을 확률적으로 적용
                persona_strength_modifier = apply_persona_influence(
                    1.0, (emp1_collab + emp2_collab) / 2, 0.25
                )
                
                if emp1['JobRole'] == emp2['JobRole']:
                    base_strength *= 1.3
                
                final_strength = max(0.1, min(1.0, base_strength * persona_strength_modifier * np.random.uniform(0.7, 1.3)))
                
                if final_strength >= 0.4:
                    # 관계 품질 결정
                    poor_prob = apply_persona_influence((emp1_quality + emp2_quality) / 2, 1.0, 0.2)
                    
                    if np.random.random() < poor_prob:
                        quality_options = ['fair', 'poor']
                    else:
                        quality_options = ['excellent', 'good', 'fair']
                    
                    # 상호작용 빈도
                    freq_modifier = apply_persona_influence(
                        1.0, (emp1_freq + emp2_freq) / 2, 0.2
                    )
                    
                    collaboration_info = {
                        'colleague_id': emp2_id,
                        'colleague_name': f"Employee_{emp2_id}",
                        'colleague_role': emp2['JobRole'],
                        'colleague_level': int(emp2['JobLevel']),
                        'collaboration_strength': round(final_strength, 3),
                        'interaction_frequency': round(np.random.beta(2, 5) * freq_modifier, 3),
                        'collaboration_type': 'same_department',
                        'common_projects': np.random.randint(1, 4),
                        'relationship_quality': np.random.choice(quality_options)
                    }
                    
                    collab_rels[emp1_id].append(collaboration_info)
                    
                    # 양방향 관계 추가
                    if emp2_id not in collab_rels:
                        collab_rels[emp2_id] = []
                    
                    collab_rels[emp2_id].append({
                        'colleague_id': emp1_id,
                        'colleague_name': f"Employee_{emp1_id}",
                        'colleague_role': emp1['JobRole'],
                        'colleague_level': int(emp1['JobLevel']),
                        'collaboration_strength': round(final_strength, 3),
                        'interaction_frequency': round(np.random.beta(2, 5) * freq_modifier, 3),
                        'collaboration_type': 'same_department',
                        'common_projects': np.random.randint(1, 4),
                        'relationship_quality': np.random.choice(quality_options)
                    })
    
    return collab_rels

def generate_project_relationships(df, num_projects=25):
    """개인차가 반영된 프로젝트 팀 관계 생성"""
    project_rels = {}
    project_registry = []
    
    for project_id in range(1, num_projects + 1):
        project_code = f'PRJ_{project_id:03d}'
        
        is_cross_dept = np.random.random() < 0.4
        team_size = np.random.randint(3, 9)
        
        if is_cross_dept:
            selected_depts = np.random.choice(df['Department'].unique(), size=min(2, len(df['Department'].unique())), replace=False)
            team_members = []
            for dept in selected_depts:
                dept_employees = df[df['Department'] == dept]
                dept_team_size = max(1, team_size // len(selected_depts))
                if len(dept_employees) > 0:
                    selected = dept_employees.sample(n=min(dept_team_size, len(dept_employees)), random_state=RNG_SEED + project_id)
                    team_members.append(selected)
            
            if team_members:
                team = pd.concat(team_members)
                project_type = 'cross_department'
            else:
                continue
        else:
            dept = np.random.choice(df['Department'].unique())
            dept_employees = df[df['Department'] == dept]
            team = dept_employees.sample(n=min(team_size, len(dept_employees)), random_state=RNG_SEED + project_id)
            project_type = 'single_department'
        
        project_info = {
            'project_id': project_code,
            'project_name': f'Project_{project_code}',
            'project_type': project_type,
            'team_size': len(team),
            'start_date': '2023-01-15',
            'end_date': '2023-12-15',
            'status': 'active' if np.random.random() < 0.8 else 'completed',
            'priority': np.random.choice(['high', 'medium', 'low'])
        }
        project_registry.append(project_info)
        
        team_list = list(team['EmployeeNumber'].astype(str))
        
        # 프로젝트 리더 선정 - 개인별 리더십 성향 반영
        leadership_scores = []
        for _, emp in team.iterrows():
            emp_id = str(emp['EmployeeNumber'])
            persona = emp.get('argmax_Persona_Code', 'N01')
            
            # 개인별 리더십 성향 + 확률적 변동
            individual_leadership = get_persona_bias(persona, emp_id, 'leadership_prob')
            leadership_with_randomness = apply_persona_influence(individual_leadership, 1.0, 0.3)
            
            # 직급과 개인별 리더십 성향을 조합
            leadership_score = emp['JobLevel'] * 0.6 + leadership_with_randomness * 0.4
            leadership_scores.append((emp['EmployeeNumber'], leadership_score))
        
        leadership_scores.sort(key=lambda x: x[1], reverse=True)
        leader_candidates = [str(emp_id) for emp_id, _ in leadership_scores[:max(1, len(leadership_scores)//3)]]
        
        for _, emp in team.iterrows():
            emp_id = str(emp['EmployeeNumber'])
            emp_persona = emp.get('argmax_Persona_Code', 'N01')
            
            # 개인별 프로젝트 참여 성향
            emp_collab = get_persona_bias(emp_persona, emp_id, 'collab_strength')
            emp_leadership = get_persona_bias(emp_persona, emp_id, 'leadership_prob')
            
            # 때로는 참여하지 않을 수도 있음 (개인차)
            participation_prob = apply_persona_influence(0.8, emp_collab, 0.2)
            if np.random.random() > participation_prob:
                continue  # 이 직원은 참여하지 않음
            
            if emp_id not in project_rels:
                project_rels[emp_id] = []
            
            teammates = [tid for tid in team_list if tid != emp_id]
            
            # 역할 할당 - 개인별 리더십 성향의 확률적 반영
            leadership_tendency = apply_persona_influence(emp_leadership, 1.0, 0.3)
            
            if emp_id in leader_candidates and leadership_tendency > 0.6:
                role_options = ['lead', 'specialist']
                role_weights = [0.7, 0.3]
            elif leadership_tendency > 0.4:
                role_options = ['contributor', 'specialist'] 
                role_weights = [0.6, 0.4]
            else:
                role_options = ['contributor', 'support']
                role_weights = [0.7, 0.3]
            
            assigned_role = np.random.choice(role_options, p=role_weights)
            
            # 기여도도 개인별 협업 성향 반영
            contribution_base = np.random.uniform(0.5, 1.0)
            contribution_influenced = apply_persona_influence(contribution_base, emp_collab, 0.25)
            
            project_rels[emp_id].append({
                'project_id': project_code,
                'project_name': project_info['project_name'],
                'project_type': project_type,
                'role_in_project': assigned_role,
                'teammates': [
                    {
                        'teammate_id': tid,
                        'teammate_name': f'Employee_{tid}',
                        'collaboration_intensity': round(np.random.uniform(0.6, 1.0), 3)
                    }
                    for tid in teammates[:3]
                ],
                'project_start': project_info['start_date'],
                'project_end': project_info['end_date'],
                'project_status': project_info['status'],
                'contribution_level': round(max(0.3, min(1.0, contribution_influenced)), 3)
            })
    
    return project_rels, project_registry

print("개인차 반영 모든 관계 생성 함수들 정의 완료")

관계 데이터 생성 실행

In [ ]:
# ------------------------------------------------------
# 은밀한 페르소나 영향이 반영된 관계 데이터 생성 실행
# ------------------------------------------------------

# 데이터 로딩
df = pd.read_csv(DATA_PATH)
print(f"데이터 로딩 완료: {len(df)}명의 직원")

# 페르소나 분포 확인 (내부 분석용)
persona_counts = df['argmax_Persona_Code'].value_counts()
print(f"\n내부 페르소나 분포 (관계 데이터에는 미노출):")
persona_name_mapping = {
    'P01': '번아웃에 직면한 직원',
    'P02': '온보딩에 실패한 직원', 
    'P03': '성장이 정체된 직원',
    'P04': '저평가된 직원',
    'S01': '안정적인 핵심인재',
    'S02': '라이징 스타',
    'S03': '내재적 동기가 높은 직원',
    'N01': '현상만 유지하는 직원',
    'N02': '유능하지만 불만이 많은 직원',
    'N03': '신규 부모'
}

for persona, count in persona_counts.items():
    persona_name = persona_name_mapping.get(persona, persona)
    print(f"- {persona} ({persona_name}): {count}명")

# 각 관계 유형별 데이터 생성 (페르소나 영향은 내부적으로만 적용)
print(f"\n관계 데이터 생성 중 (페르소나 영향 은밀 반영)...")

# 1. 계층 관계 - 관리 스타일, 보고 빈도에 페르소나 특성이 은밀하게 반영
hierarchy_relationships = generate_hierarchy_relationships(df)
print(f"계층 관계 생성 완료: {len(hierarchy_relationships)}명의 관계 정보")

# 2. 협업 관계 - 협업 강도, 빈도, 품질에 페르소나 편향 반영
collaboration_relationships = generate_collaboration_relationships(df)
print(f"협업 관계 생성 완료: {len(collaboration_relationships)}명의 관계 정보")

# 3. 프로젝트 관계 - 역할 할당, 기여도에 리더십 성향 반영
project_relationships, project_registry = generate_project_relationships(df, num_projects=25)
print(f"프로젝트 관계 생성 완료: {len(project_relationships)}명의 관계 정보")
print(f"프로젝트 등록: {len(project_registry)}개 프로젝트")

# 관계 통계
total_hierarchy_relations = sum(len(rels['manages']) for rels in hierarchy_relationships.values())
total_collaboration_relations = sum(len(rels) for rels in collaboration_relationships.values())
total_project_relations = sum(len(rels) for rels in project_relationships.values())

print(f"\n생성된 관계 통계:")
print(f"- 계층 관계: {total_hierarchy_relations}개")
print(f"- 협업 관계: {total_collaboration_relations}개")
print(f"- 프로젝트 관계: {total_project_relations}개")

# 페르소나 영향 검증 (내부 분석용 - 실제 데이터에는 미포함)
print(f"\n은밀한 페르소나 영향 검증:")

# 협업 강도 분석
print("협업 강도 패턴:")
for persona_code in ['P01', 'P02', 'S01', 'S02', 'N01', 'N02']:
    if persona_code not in persona_counts.index:
        continue
        
    persona_employees = df[df['argmax_Persona_Code'] == persona_code]['EmployeeNumber'].astype(str)
    
    # 해당 페르소나의 협업 강도 수집
    persona_strengths = []
    persona_qualities = []
    
    for emp_id in persona_employees:
        if emp_id in collaboration_relationships:
            for rel in collaboration_relationships[emp_id]:
                persona_strengths.append(rel['collaboration_strength'])
                persona_qualities.append(rel['relationship_quality'])
    
    if persona_strengths:
        avg_strength = np.mean(persona_strengths)
        poor_quality_rate = sum(1 for q in persona_qualities if q in ['poor', 'fair']) / len(persona_qualities)
        
        expected_bias = get_persona_bias(persona_code)
        print(f"- {persona_code}: 평균 강도 {avg_strength:.3f} (편향: {expected_bias['collab_strength']:.1f}), "
              f"품질 저하율 {poor_quality_rate:.3f} (예상: {expected_bias['quality_poor_prob']:.1f})")

# 프로젝트 리더십 역할 분석
print(f"\n리더십 역할 분포:")
for persona_code in ['P01', 'P02', 'S01', 'S02', 'N01', 'N02']:
    if persona_code not in persona_counts.index:
        continue
        
    persona_employees = df[df['argmax_Persona_Code'] == persona_code]['EmployeeNumber'].astype(str)
    
    total_roles = 0
    lead_roles = 0
    
    for emp_id in persona_employees:
        if emp_id in project_relationships:
            for project in project_relationships[emp_id]:
                total_roles += 1
                if project['role_in_project'] in ['lead', 'specialist']:
                    lead_roles += 1
    
    if total_roles > 0:
        leadership_rate = lead_roles / total_roles
        expected_leadership = get_persona_bias(persona_code)['leadership_prob']
        print(f"- {persona_code}: 리더십 역할 비율 {leadership_rate:.3f} (예상: {expected_leadership:.1f})")

print(f"\n✅ 관계 데이터 생성 완료!")
print(f"   - 페르소나 정보는 관계 데이터에 직접 노출되지 않음")
print(f"   - 페르소나별 행동 패턴은 통계적 편향으로 은밀하게 반영됨")
print(f"   - Cognita 에이전트가 이러한 패턴을 분석하여 위험도를 추론해야 함")

XML 구조 생성

In [ ]:
# ------------------------------------------------------
# XML 구조 생성 및 데이터 입력 (페르소나/위험도 정보 제외)
# ------------------------------------------------------

def create_employee_relationships_xml(df, hierarchy_rels, collab_rels, project_rels, project_registry):
    """직원별 관계 정보를 XML로 생성 (페르소나 정보 미포함)"""
    
    # 루트 엘리먼트 생성
    root = ET.Element("OrganizationNetwork")
    root.set("generated_date", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    root.set("total_employees", str(len(df)))
    root.set("data_source", "IBM_HR_personas_assigned.csv")
    
    # 프로젝트 레지스트리 섹션
    projects_section = ET.SubElement(root, "ProjectRegistry")
    for project in project_registry:
        project_elem = ET.SubElement(projects_section, "Project")
        for key, value in project.items():
            project_elem.set(key, str(value))
    
    # 직원 관계 섹션
    employees_section = ET.SubElement(root, "Employees")
    
    for _, emp in df.iterrows():
        emp_id = str(emp['EmployeeNumber'])
        
        # 직원 기본 정보 (페르소나/위험도 정보 제외)
        employee_elem = ET.SubElement(employees_section, "Employee")
        employee_elem.set("id", emp_id)
        employee_elem.set("name", f"Employee_{emp_id}")
        employee_elem.set("department", str(emp['Department']))
        employee_elem.set("job_role", str(emp['JobRole']))
        employee_elem.set("job_level", str(emp['JobLevel']))
        employee_elem.set("years_at_company", str(emp['YearsAtCompany']))
        # 페르소나 및 위험도 정보는 Cognita가 추론해야 하므로 제외
        # employee_elem.set("risk_tier", str(emp['argmax_RiskTier']))    # 제거
        # employee_elem.set("persona", str(emp['argmax_Persona']))       # 제거
        
        # 계층 관계 (Hierarchy Relationships)
        hierarchy_elem = ET.SubElement(employee_elem, "HierarchyRelationships")
        
        if emp_id in hierarchy_rels:
            hierarchy_info = hierarchy_rels[emp_id]
            
            # 상사 정보
            if hierarchy_info['reports_to']:
                manager_elem = ET.SubElement(hierarchy_elem, "ReportsTo")
                for key, value in hierarchy_info['reports_to'].items():
                    manager_elem.set(key, str(value))
            
            # 부하직원 정보
            if hierarchy_info['manages']:
                subordinates_elem = ET.SubElement(hierarchy_elem, "Manages")
                for subordinate in hierarchy_info['manages']:
                    sub_elem = ET.SubElement(subordinates_elem, "Subordinate")
                    for key, value in subordinate.items():
                        sub_elem.set(key, str(value))
        
        # 협업 관계 (Collaboration Relationships)
        collab_elem = ET.SubElement(employee_elem, "CollaborationRelationships")
        
        if emp_id in collab_rels:
            for colleague in collab_rels[emp_id]:
                colleague_elem = ET.SubElement(collab_elem, "Colleague")
                for key, value in colleague.items():
                    colleague_elem.set(key, str(value))
        
        # 프로젝트 관계 (Project Relationships)
        project_elem = ET.SubElement(employee_elem, "ProjectRelationships")
        
        if emp_id in project_rels:
            for project in project_rels[emp_id]:
                proj_elem = ET.SubElement(project_elem, "ProjectParticipation")
                
                # 프로젝트 기본 정보
                for key, value in project.items():
                    if key != 'teammates':
                        proj_elem.set(key, str(value))
                
                # 팀동료 정보
                if 'teammates' in project:
                    teammates_elem = ET.SubElement(proj_elem, "Teammates")
                    for teammate in project['teammates']:
                        teammate_elem = ET.SubElement(teammates_elem, "Teammate")
                        for key, value in teammate.items():
                            teammate_elem.set(key, str(value))
    
    return root

# XML 생성 실행
print("XML 구조 생성 중 (페르소나/위험도 정보 제외)...")
xml_root = create_employee_relationships_xml(
    df, hierarchy_relationships, collaboration_relationships, 
    project_relationships, project_registry
)

print("XML 구조 생성 완료")
print("- 기본 HR 정보 (부서, 직급, 근속연수) 포함")
print("- 페르소나 및 위험도 정보 제외 (Cognita가 관계 패턴으로 추론해야 함)")

XML 및 JSON 파일 저장

In [ ]:
# ------------------------------------------------------
# XML 및 JSON 파일 저장 (페르소나/위험도 정보 제외)
# ------------------------------------------------------

def save_xml_file(root_element, file_path):
    """XML 파일을 보기 좋게 포매팅해서 저장"""
    # 예쁜 포매팅으로 저장
    xml_str = prettify_xml(root_element)
    
    # UTF-8 인코딩으로 저장
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(xml_str)
    
    print(f"XML 파일 저장 완료: {file_path}")

def create_json_summary(df, hierarchy_rels, collab_rels, project_rels, project_registry):
    """JSON 형태의 요약 데이터 생성 (페르소나 정보 미포함)"""
    json_data = {
        "metadata": {
            "generated_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_employees": len(df),
            "total_projects": len(project_registry),
            "data_source": "IBM_HR_personas_assigned.csv"
        },
        "projects": project_registry,
        "employee_relationships": {}
    }
    
    for _, emp in df.iterrows():
        emp_id = str(emp['EmployeeNumber'])
        
        employee_data = {
            "basic_info": {
                "name": f"Employee_{emp_id}",
                "department": emp['Department'],
                "job_role": emp['JobRole'],
                "job_level": int(emp['JobLevel']),
                "years_at_company": int(emp['YearsAtCompany'])
                # 페르소나 및 위험도 정보는 Cognita가 추론해야 하므로 제외
                # "risk_tier": emp['argmax_RiskTier'],    # 제거
                # "persona": emp['argmax_Persona']        # 제거
            },
            "relationships": {
                "hierarchy": hierarchy_rels.get(emp_id, {"manages": [], "reports_to": None}),
                "collaboration": collab_rels.get(emp_id, []),
                "projects": project_rels.get(emp_id, [])
            }
        }
        
        json_data["employee_relationships"][emp_id] = employee_data
    
    return json_data

# XML 파일 저장
save_xml_file(xml_root, XML_OUTPUT_PATH)

# JSON 파일도 생성 및 저장
json_data = create_json_summary(
    df, hierarchy_relationships, collaboration_relationships,
    project_relationships, project_registry
)

with open(JSON_OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print(f"JSON 파일 저장 완료: {JSON_OUTPUT_PATH}")

# 파일 크기 확인
xml_size = XML_OUTPUT_PATH.stat().st_size / (1024 * 1024)  # MB
json_size = JSON_OUTPUT_PATH.stat().st_size / (1024 * 1024)  # MB

print(f"\n파일 크기:")
print(f"- XML 파일: {xml_size:.2f} MB")
print(f"- JSON 파일: {json_size:.2f} MB")
print(f"\n데이터 구성:")
print(f"- 기본 HR 정보 포함: 부서, 직급, 근속연수")
print(f"- 페르소나/위험도 정보 제외: Cognita 추론 대상")
print(f"- 관계 데이터 포함: 계층, 협업, 프로젝트")

XML 데이터 검증 및 미리보기

In [ ]:
# ------------------------------------------------------
# 생성된 XML 데이터 검증 및 미리보기 (페르소나/위험도 정보 제외)
# ------------------------------------------------------

def validate_and_preview_xml(xml_file_path, num_employees_to_show=3):
    """생성된 XML 파일 검증 및 미리보기"""
    
    try:
        # XML 파일 파싱
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        print(f"=== XML 파일 검증 결과 ===")
        print(f"루트 엘리먼트: {root.tag}")
        print(f"생성 일시: {root.get('generated_date')}")
        print(f"총 직원 수: {root.get('total_employees')}")
        
        # 프로젝트 레지스트리 확인
        projects_section = root.find('ProjectRegistry')
        if projects_section is not None:
            project_count = len(projects_section.findall('Project'))
            print(f"등록된 프로젝트: {project_count}개")
        
        # 직원 섹션 확인
        employees_section = root.find('Employees')
        if employees_section is not None:
            employees = employees_section.findall('Employee')
            print(f"XML에 포함된 직원: {len(employees)}명")
            
            # 샘플 직원들의 관계 정보 미리보기
            print(f"\n=== 샘플 직원 관계 정보 (상위 {num_employees_to_show}명) ===")
            
            for i, emp_elem in enumerate(employees[:num_employees_to_show]):
                emp_id = emp_elem.get('id')
                emp_name = emp_elem.get('name')
                dept = emp_elem.get('department')
                role = emp_elem.get('job_role')
                job_level = emp_elem.get('job_level')
                years_at_company = emp_elem.get('years_at_company')
                
                print(f"\n[직원 {i+1}: {emp_name}]")
                print(f"  부서: {dept}, 직무: {role}, 직급: Level {job_level}")
                print(f"  근속연수: {years_at_company}년")
                # 위험도와 페르소나 정보는 제거됨 (Cognita가 추론해야 함)
                
                # 계층 관계
                hierarchy = emp_elem.find('HierarchyRelationships')
                if hierarchy is not None:
                    reports_to = hierarchy.find('ReportsTo')
                    manages = hierarchy.find('Manages')
                    
                    if reports_to is not None:
                        mgr_id = reports_to.get('manager_id')
                        mgr_role = reports_to.get('manager_role')
                        reporting_freq = reports_to.get('reporting_frequency')
                        print(f"  상사: {mgr_id} ({mgr_role}), 보고빈도: {reporting_freq}")
                    else:
                        print(f"  상사: 없음 (최고 관리층)")
                    
                    if manages is not None:
                        subordinates = manages.findall('Subordinate')
                        print(f"  부하직원: {len(subordinates)}명")
                        if subordinates:
                            # 관리 스타일 샘플 출력
                            sample_mgmt_style = subordinates[0].get('management_style')
                            print(f"    관리 스타일: {sample_mgmt_style}")
                
                # 협업 관계
                collab = emp_elem.find('CollaborationRelationships')
                if collab is not None:
                    colleagues = collab.findall('Colleague')
                    if colleagues:
                        # 협업 강도별 분석
                        strong_collabs = [c for c in colleagues if float(c.get('collaboration_strength', 0)) > 0.7]
                        poor_quality = [c for c in colleagues if c.get('relationship_quality') in ['poor', 'fair']]
                        
                        print(f"  협업 동료: {len(colleagues)}명 (강한 협업: {len(strong_collabs)}명)")
                        if poor_quality:
                            print(f"    관계 품질 저하: {len(poor_quality)}명")
                
                # 프로젝트 참여
                projects = emp_elem.find('ProjectRelationships')
                if projects is not None:
                    participations = projects.findall('ProjectParticipation')
                    active_projects = [p for p in participations if p.get('project_status') == 'active']
                    lead_roles = [p for p in participations if p.get('role_in_project') in ['lead', 'specialist']]
                    
                    print(f"  참여 프로젝트: {len(participations)}개 (활성: {len(active_projects)}개)")
                    if lead_roles:
                        print(f"    리더십 역할: {len(lead_roles)}개")
        
        print(f"\n=== 데이터 구성 확인 ===")
        print(f"✅ 기본 HR 정보: 부서, 직급, 근속연수 포함")
        print(f"✅ 관계 패턴: 계층, 협업, 프로젝트 관계 포함")
        print(f"❌ 페르소나/위험도: 의도적으로 제외 (AI 추론 대상)")
        
        print(f"\n=== XML 파일 검증 완료 ===")
        return True
        
    except Exception as e:
        print(f"XML 파일 검증 실패: {str(e)}")
        return False

# XML 검증 실행
validation_success = validate_and_preview_xml(XML_OUTPUT_PATH, num_employees_to_show=5)

if validation_success:
    print(f"\n✅ XML 관계 데이터 파일이 성공적으로 생성되었습니다!")
    print(f"   파일 위치: {XML_OUTPUT_PATH}")
    print(f"   JSON 버전: {JSON_OUTPUT_PATH}")
    print(f"   Cognita 에이전트가 관계 패턴을 분석하여 위험도를 추론해야 함")
else:
    print(f"\n❌ XML 파일 생성에 문제가 있습니다.")

특정 직원의 관계 조회 함수

In [ ]:
# ------------------------------------------------------
# 특정 직원의 관계 조회 함수 (페르소나/위험도 정보 제외)
# ------------------------------------------------------

def query_employee_relationships(xml_file_path, employee_id):
    """특정 직원의 모든 관계 정보 조회 (Cognita 관점에서)"""
    
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        # 해당 직원 찾기
        employees_section = root.find('Employees')
        target_employee = None
        
        for emp in employees_section.findall('Employee'):
            if emp.get('id') == str(employee_id):
                target_employee = emp
                break
        
        if target_employee is None:
            print(f"직원 ID {employee_id}를 찾을 수 없습니다.")
            return None
        
        # 직원 기본 정보 (Cognita가 볼 수 있는 정보)
        print(f"=== 직원 {employee_id} 관계 정보 ===")
        print(f"이름: {target_employee.get('name')}")
        print(f"부서: {target_employee.get('department')}")
        print(f"직무: {target_employee.get('job_role')}")
        print(f"직급: Level {target_employee.get('job_level')}")
        print(f"근속: {target_employee.get('years_at_company')}년")
        # 페르소나와 위험도는 제거 - Cognita가 아래 패턴들로 추론해야 함
        
        # 계층 관계
        hierarchy = target_employee.find('HierarchyRelationships')
        if hierarchy is not None:
            print(f"\n📋 계층 관계:")
            
            reports_to = hierarchy.find('ReportsTo')
            if reports_to is not None:
                print(f"  상사: {reports_to.get('manager_id')} ({reports_to.get('manager_role')})")
                print(f"       보고 빈도: {reports_to.get('reporting_frequency')}")
            else:
                print(f"  상사: 없음 (최고 관리층)")
            
            manages = hierarchy.find('Manages')
            if manages is not None:
                subordinates = manages.findall('Subordinate')
                print(f"  부하직원: {len(subordinates)}명")
                if subordinates:
                    mgmt_style = subordinates[0].get('management_style')
                    print(f"    관리 스타일: {mgmt_style}")
                for sub in subordinates[:3]:  # 상위 3명만 표시
                    print(f"    - {sub.get('employee_id')} ({sub.get('job_role')})")
        
        # 협업 관계 - 상세 패턴 분석
        collab = target_employee.find('CollaborationRelationships')
        if collab is not None:
            colleagues = collab.findall('Colleague')
            print(f"\n🤝 협업 관계 패턴 분석: {len(colleagues)}명")
            
            if colleagues:
                # 협업 강도 통계
                strengths = [float(c.get('collaboration_strength', 0)) for c in colleagues]
                avg_strength = sum(strengths) / len(strengths)
                strong_collabs = [c for c in colleagues if float(c.get('collaboration_strength', 0)) > 0.6]
                
                # 관계 품질 분석
                poor_relations = [c for c in colleagues if c.get('relationship_quality') in ['poor', 'fair']]
                excellent_relations = [c for c in colleagues if c.get('relationship_quality') == 'excellent']
                
                print(f"  평균 협업 강도: {avg_strength:.3f}")
                print(f"  강한 협업 관계: {len(strong_collabs)}명 ({len(strong_collabs)/len(colleagues)*100:.1f}%)")
                print(f"  품질 저하 관계: {len(poor_relations)}명 ({len(poor_relations)/len(colleagues)*100:.1f}%)")
                print(f"  우수한 관계: {len(excellent_relations)}명 ({len(excellent_relations)/len(colleagues)*100:.1f}%)")
                
                # 주요 협업 동료 (상위 5명)
                if strong_collabs:
                    print(f"  주요 협업 동료:")
                    for colleague in strong_collabs[:5]:
                        strength = colleague.get('collaboration_strength')
                        quality = colleague.get('relationship_quality')
                        freq = colleague.get('interaction_frequency')
                        print(f"    - {colleague.get('colleague_id')} ({colleague.get('colleague_role')}) "
                              f"강도: {strength}, 품질: {quality}, 빈도: {freq}")
        
        # 프로젝트 참여 - 리더십 및 기여 패턴 분석
        projects = target_employee.find('ProjectRelationships')
        if projects is not None:
            participations = projects.findall('ProjectParticipation')
            active_projects = [p for p in participations if p.get('project_status') == 'active']
            
            print(f"\n🚀 프로젝트 참여 패턴: {len(participations)}개")
            
            if participations:
                # 역할 분석
                lead_roles = [p for p in participations if p.get('role_in_project') in ['lead', 'specialist']]
                support_roles = [p for p in participations if p.get('role_in_project') == 'support']
                
                # 기여도 분석
                contributions = [float(p.get('contribution_level', 0)) for p in participations]
                avg_contribution = sum(contributions) / len(contributions) if contributions else 0
                
                print(f"  리더십 역할: {len(lead_roles)}개 ({len(lead_roles)/len(participations)*100:.1f}%)")
                print(f"  지원 역할: {len(support_roles)}개 ({len(support_roles)/len(participations)*100:.1f}%)")
                print(f"  평균 기여도: {avg_contribution:.3f}")
                print(f"  활성 프로젝트: {len(active_projects)}개")
                
                if active_projects:
                    print(f"  현재 활성 프로젝트:")
                    for project in active_projects:
                        proj_id = project.get('project_id')
                        role = project.get('role_in_project')
                        contribution = project.get('contribution_level')
                        project_type = project.get('project_type')
                        print(f"    - {proj_id} (역할: {role}, 기여도: {contribution}, 유형: {project_type})")
        
        # 패턴 기반 위험 신호 힌트 (Cognita가 학습해야 할 패턴들)
        print(f"\n🔍 관찰되는 행동 패턴:")
        
        # 협업 패턴
        if collab is not None and colleagues:
            if avg_strength < 0.5:
                print(f"  - 협업 강도가 평균보다 낮음 ({avg_strength:.3f})")
            if len(poor_relations) / len(colleagues) > 0.4:
                print(f"  - 관계 품질 저하 비율이 높음 ({len(poor_relations)/len(colleagues)*100:.1f}%)")
        
        # 리더십 패턴
        if projects is not None and participations:
            if len(lead_roles) == 0:
                print(f"  - 리더십 역할을 맡지 않음")
            elif len(lead_roles) / len(participations) > 0.7:
                print(f"  - 높은 리더십 역할 비율 ({len(lead_roles)/len(participations)*100:.1f}%)")
            
            if avg_contribution < 0.6:
                print(f"  - 프로젝트 기여도가 낮음 ({avg_contribution:.3f})")
        
        # 계층 관계 패턴
        if hierarchy is not None:
            if reports_to is not None:
                freq = reports_to.get('reporting_frequency')
                if freq == 'daily':
                    print(f"  - 상사와의 잦은 보고 (매일)")
        
        print(f"\n💡 Cognita는 이러한 패턴들을 종합하여 직원의 상태를 추론해야 합니다.")
        
        return target_employee
        
    except Exception as e:
        print(f"관계 조회 실패: {str(e)}")
        return None

# 사용 예시: 랜덤한 직원 3명의 관계 조회
sample_employee_ids = df['EmployeeNumber'].sample(3).tolist()

for emp_id in sample_employee_ids:
    query_employee_relationships(XML_OUTPUT_PATH, emp_id)
    print("\n" + "="*60 + "\n")